In [ ]:
import glob
import os
from google.colab import drive
drive.mount('/content/drive')
path = r"/content/drive/My Drive/Colab Notebooks/NN" 
all_files = glob.glob(path + "/*.csv")
all_files
'''
all_files values would iterate throught the following file names:
att_amalgam_day.csv
ver_amalgam_day.csv
spri_amalgam_day.csv
tmo_amalgam_day.csv
'''

In [9]:
import pandas as pd
def get_col(filename,column):
  inputs = ['norm_score','AT&T-US', 'SPRINT-US', 'TMOBILE-US', 'VERIZON-US']
  inputs.append(column)
  data = pd.read_csv(filename)
  return data[inputs]

colName = 'Operating Income to Assets'
'''
colName will be assigned the following values:
'Return On Assets', 
'Return On Sales', 
'Operating Income to Sales',
'Operating Income to Assets', 
'Operating Epenses to Sales', 
'Cost of Goods to Sales',
'Sellecing, General Administrative Expenses to Sales']
'''
data = []
for csv in all_files:
  data.append(get_col(csv,colName))
#data[0]

In [11]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
import time, os, fnmatch, shutil
from sklearn.model_selection import train_test_split

numpy.random.seed(7)
testSize=0.8
cv = 3

# Function to create model, required for KerasClassifier
def create_model(neurons=15):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=5, activation='relu',kernel_initializer='normal'))
	model.add(Dense(1))
	# Compile model
	model.compile(loss='mse',optimizer='adam',metrics=['accuracy','mae'])
	return model

In [ ]:
for i in range(len(data)):
  train, test = train_test_split(data[i], test_size=testSize)
  # split into input (X) and output (Y) variables
  xTrain = train.loc[:,:'VERIZON-US']
  yTrain = train.loc[:,colName:]
  xTest = test.loc[:,:'VERIZON-US']
  yTest = test.loc[:,colName:]

  # create model
  model = KerasRegressor(build_fn=create_model,verbose=0)
  neurons = [10,15,25,30,40,64]
  # define the grid search parameters
  batch_size =[5,10,15,25,30]
  epochs = [100,150]

  param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs)
  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv)

  grid_result = grid.fit(xTrain, yTrain)
  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  #grid_result.cv_results_['param_neurons']

  t = time.localtime()
  timestamp = time.strftime('%b-%d-%Y_%H%M', t)
  tmplst = []
  tmplst.append(colName)
  tmplst.append("Result/Best Score:" + str(grid_result.best_score_))
  tmplst.append("Best Parameters:" + str(grid_result.best_params_))
  tmplst.append("Parameters Used:" + str(param_grid))
  tmplst.append("Cross Validation Folds:"+ str(cv))
  
  for mean, stdev, param in zip(means, stds, params):
      #print("%f (%f) with: %r" % (mean, stdev, param))
      tmplst.append("%f (%f) with: %r" % (mean, stdev, param))
  df = pd.DataFrame(tmplst)
  filename = r'/content/drive/My Drive/Colab Notebooks/RESULTS/neurons_%s_%s_%s_po.csv'%(colName,timestamp,str(i))
  df.to_csv(filename,encoding='utf-8')



[Ref.1 -Ch.3 Manning Book: Deep Learning w/Python](https://livebook.manning.com/book/deep-learning-with-python/chapter-3/297)

[Ref.2 Tutorial:NN-Python-Keras](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

[Ref.3 Adam Optimization for Deep Learning](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

[Ref.4 Normalize Columns-Pandas DF](https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame)

[Ref.5 GridSearch-hyperparameters-keras/](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)

[Ref.6 Making Predictions](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

[Ref.7 Regression Prediction](https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/)

[ref.8 Regresson Tutorial](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

[Ref.9 Metrics , Regression Tutorial](https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/)

[Ref.10 StackerOverflow: Perform prediction keraregressor](#https://stackoverflow.com/questions/44132652/keras-how-to-perform-a-prediction-using-kerasregressor)